In [1]:
import pickle

from HH4b.utils import ShapeVar, get_var_mapping, singleVarHist
from HH4b import hh_vars, plotting, postprocessing, run_utils
from collections import OrderedDict

from pathlib import Path
import hist
from hist.intervals import ratio_uncertainty
import numpy as np

# load pickle
with open("hist_2022-2023.pkl", "rb") as f:
    loaded_hists = pickle.load(f)

control_plot_vars = [
    ShapeVar(
        var="bdt_score_finebin",
        label=r"D(ggHH-vs-bkg)",
        bins=[0, 0.0299999, 0.7549999, 0.9399999, 1],
        reg=False,
    ),
    ShapeVar(
        var="bdt_score_vbf_finebin",
        label=r"D(qqHH-vs-bkg)",
        bins=[0, 0.0299999, 0.6374999, 0.9666666, 1],
        reg=False,
    ),
]

samples = ["data", "qcd", "ttbar", "dibosonvjets"]
sample_dict = OrderedDict(
    [
        ("qcd", "qcd"),
        ("dibosonvjets", ["diboson", "vjets"]),
        ("ttbar", "ttbar"),
        ("data", "data"),
    ]
)

data_key = "data"

In [2]:
hists = {}
for shape_var in control_plot_vars:
    hists[shape_var.var] = hist.Hist(
        hist.axis.StrCategory(samples, name="Sample"),
        shape_var.axis,
        storage="double",
    )

    h = hists[shape_var.var]
    loaded_h = loaded_hists[shape_var.var]
    for key, file_key in sample_dict.items():
        if key != data_key:
            if isinstance(file_key, list):
                # sum components
                values = None
                for fk in file_key:
                    if values is None:
                        values = loaded_h[fk, :].values()
                    else:
                        values += loaded_h[fk, :].values()
            else:
                values = loaded_h[file_key, :].values()
            data_key_index = np.where(np.array(list(h.axes[0])) == key)[0][0]
            h.view(flow=False)[data_key_index, :] = values

    data_key_index = np.where(np.array(list(h.axes[0])) == data_key)[0][0]
    h.view(flow=False)[data_key_index, :] = np.nan_to_num(loaded_h[data_key, :].values())

In [4]:
year = "2022-2023"
plot_dir = Path(f"../../../plots/PostProcess/14FMar25_60")
plot_dir.mkdir(exist_ok=True, parents=True)
tag = "14Dec24_hadronic_ttbar_val_v12v2_private_signal"
odir = f"control_{tag}/{year}"
(plot_dir / odir).mkdir(exist_ok=True, parents=True)

for shape_var in control_plot_vars:

    num = (
        hists[shape_var.var]["data", :].values()
        - sum([hists[shape_var.var][bkg, :] for bkg in ["qcd", "dibosonvjets"]]).values()
    )
    den = hists[shape_var.var]["ttbar", :].values()

    sf_values = num / den
    sf_err = ratio_uncertainty(num, den, "poisson")

    plotting.ratioHistPlot(
        hists[shape_var.var],
        year,
        [],
        ["qcd", "ttbar", "dibosonvjets"],
        title=r"$t\bar{t}$ fully hadronic CR",
        name=f"{plot_dir}/{odir}/{shape_var.var}",
        show=False,
        log=True,
        bg_order=["dibosonvjets", "qcd", "ttbar"],
        plot_significance=False,
        significance_dir=shape_var.significance_dir,
        ratio_ylims=[0, 2],
        bg_err_mcstat=False,
        reweight_qcd=False,
        save_pdf=True,
        ylim=1e7,
        ylim_low=1e-1,
        xlim=1,
        xlim_low=0,
        ratio=sf_values,
        ratio_err=sf_err,
        ratio_label=r"$t\bar{t}$ SF",
        cms_label="Supplementary",
    )